In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.getenv("FINNHUB_API_KEY"))


d12gec1r01qmhi3igcegd12gec1r01qmhi3igcf0


In [ ]:
import os
import requests
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, DoubleType, BooleanType
from pyspark.sql.functions import col
import time

import pandas as pd

# Load .env file
load_dotenv()

# Read API key from environment
API_KEY = os.getenv("FINNHUB_API_KEY")
BASE_URL = "https://finnhub.io/api/v1"

def get_symbols():
    url =f"{BASE_URL}/stock/symbol"
    params ={
        "exchange" : "US",
        "token": API_KEY
    }
    response =requests.get(url,params=params)
    return response.json()

get_symbols()



[{'currency': 'USD',
  'description': 'MUNTERS GROUP AB',
  'displaySymbol': 'MMNNF',
  'figi': 'BBG00H2V4B20',
  'isin': None,
  'mic': 'OOTC',
  'shareClassFIGI': 'BBG00GLM1NN5',
  'symbol': 'MMNNF',
  'symbol2': '',
  'type': 'Common Stock'},
 {'currency': 'USD',
  'description': 'TRUE CORP PCL/NEW-FOREIGN',
  'displaySymbol': 'TCPFF',
  'figi': 'BBG01GF7R9M8',
  'isin': None,
  'mic': 'OOTC',
  'shareClassFIGI': 'BBG01FPCJ4Y5',
  'symbol': 'TCPFF',
  'symbol2': '',
  'type': 'Foreign Sh.'},
 {'currency': 'USD',
  'description': 'Federal Home Loan Mortgage Corporation',
  'displaySymbol': 'FMCKP',
  'figi': '',
  'isin': None,
  'mic': 'OOTC',
  'shareClassFIGI': '',
  'symbol': 'FMCKP',
  'symbol2': '',
  'type': ''},
 {'currency': 'USD',
  'description': 'BAKKAFROST P/F',
  'displaySymbol': 'BKFKF',
  'figi': 'BBG008F92ZX0',
  'isin': None,
  'mic': 'OOTC',
  'shareClassFIGI': 'BBG001T6XGP3',
  'symbol': 'BKFKF',
  'symbol2': '',
  'type': 'Common Stock'},
 {'currency': 'USD',
  '

In [3]:
data = get_symbols()
dataframe=pd.DataFrame(data)
#dataframe.head()
print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28742 entries, 0 to 28741
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   currency        28742 non-null  object
 1   description     28742 non-null  object
 2   displaySymbol   28742 non-null  object
 3   figi            28742 non-null  object
 4   isin            0 non-null      object
 5   mic             28742 non-null  object
 6   shareClassFIGI  28742 non-null  object
 7   symbol          28742 non-null  object
 8   symbol2         28742 non-null  object
 9   type            28742 non-null  object
dtypes: object(10)
memory usage: 2.2+ MB
None


In [4]:
spark=SparkSession.builder\
    .appName("Stock_ETL")\
    .master("local[*]")\
    .getOrCreate()

schema = StructType([
    StructField("currency", StringType(), False), 
    StructField("description", StringType(), False), 
    StructField("displaySymbol", StringType(), False), 
    StructField("figi", StringType(), False), 
    StructField("isin", StringType(), True),  
    StructField("mic", StringType(), False), 
    StructField("shareClassFIGI", StringType(), False), 
    StructField("symbol", StringType(), False), 
    StructField("symbol2", StringType(), False), 
    StructField("type", StringType(), False) 
])

'''df=spark.read(get_symbols())  -- big dataset and API call is in memory, so convert to .json file or use panndas
   Pyspark can create df only with list of dict/tuples(structured) not from standalone dict//tuple so use pandas,
   Since dtypes is object(mixed) -use explicit Schema
   column a (expects int) but gets string then it reads null, so be clear with the types (stringtype/binarytype)
'''


df=spark.createDataFrame(dataframe,schema=schema)
#df.describe().show()
df.show()


+--------+--------------------+-------------+------------+----+----+--------------+------+-------+---------------+
|currency|         description|displaySymbol|        figi|isin| mic|shareClassFIGI|symbol|symbol2|           type|
+--------+--------------------+-------------+------------+----+----+--------------+------+-------+---------------+
|     USD|             EDENRED|        EDNMF|BBG0019P5GN6|NULL|OOTC|  BBG001T6X238| EDNMF|       |   Common Stock|
|     USD|            ON24 INC|         ONTF|BBG0017YP1P8|NULL|XNYS|  BBG001TCT0B5|  ONTF|       |   Common Stock|
|     USD|LEVERAGE SHARES 2...|         ARMG|BBG01RQ6WKX1|NULL|XNAS|  BBG01RQ6WLS5|  ARMG|       |            ETP|
|     USD|TOPICUS.COM INC-S...|        TOITF|BBG00Z5JDZD9|NULL|OOTC|  BBG00XK3FSS0| TOITF|       |   Common Stock|
|     USD|ISHARES JP MORGAN...|         CEMB|BBG002WGKHN9|NULL|BATS|  BBG002WGKJD6|  CEMB|       |            ETP|
|     USD|      INDIA FUND INC|          IFN|BBG000BGRKL4|NULL|XNYS|  BBG001S7PW

In [ ]:
def get_stock_quote(symbols):
        url = f"{BASE_URL}/quote"
        params = {"symbol": symbols, "token": API_KEY}
        response = requests.get(url, params=params)
        return response.json()


In [10]:
'''list_of_symbols=dataframe['symbol'].tolist()
rows=[]
for i in list_of_symbols:
    rows.append(get_stock_quote(i))
    time.sleep(1.1)

pandas_df=pd.DataFrame(rows)
df=spark.createDataFrame(pandas_df)

'''

def get_company_profile(symbols):
    url=f"{BASE_URL}/stock/profile2"
    params={
        "symbol":symbols,
        "token":API_KEY
    }
    response=requests.get(url,params)
    return response.json()

def get_financials(symbols,metric="all"):
    url=f'{BASE_URL}/stock/metric'
    params={"symbol":symbols,"metric":metric,"token":API_KEY}
    response=requests.get(url,params=params)
    return response.json()

#executives thoughts about the stock price and valuation in the near future,monthly share purchase ratio
def insider_sentiments(symbols,fromdate,todate):
    url=f'{BASE_URL}/stock/insider-sentiment'
    params={"symbol":symbols,"from":fromdate,"to":todate,"token":API_KEY}
    response=requests.get(url,params=params)
    return response.json()

#eps_surprise is the difference between the estimated and actual earning
def eps_surprise(symbols):
    url=f'{BASE_URL}/stock/earnings'
    params={"symbol":symbols,"token":API_KEY}
    response=requests.get(url,params=params)
    return response.json()


In [ ]:
'''from datetime import datetime
symbols = input("Enter the symbol")
try:
    date1,date2=input("enter from date and to date in YYYY-MM-DD").split()
    fromdate=datetime.striptime(date1, "%Y-%m-%d").date()
    todate=datetime.striptime(date2, "%Y-%m-%d").date()

except ValueError:
    print("invalid date, enter in YYYY-MM-DD format")
    '''
symbols="AAPL"
fromdate="2024-01-01"
todate ="2024-12-31"
quote = get_stock_quote(symbols)
profile = get_company_profile(symbols)

financials = get_financials(symbols)
sentiments = insider_sentiments(symbols,fromdate,todate)
eps = eps_surprise(symbols)

#print(quote)
#print(profile)
#print(financials.keys())
#print(financials["metric"])
#print(sentiments) Visualize using graph
#print(eps) Visualize using graph

    

[{'actual': 1.65, 'estimate': 1.6596, 'period': '2025-03-31', 'quarter': 2, 'surprise': -0.0096, 'surprisePercent': -0.5785, 'symbol': 'AAPL', 'year': 2025}, {'actual': 2.4, 'estimate': 2.3964, 'period': '2024-12-31', 'quarter': 1, 'surprise': 0.0036, 'surprisePercent': 0.1502, 'symbol': 'AAPL', 'year': 2025}, {'actual': 1.64, 'estimate': 1.6343, 'period': '2024-09-30', 'quarter': 4, 'surprise': 0.0057, 'surprisePercent': 0.3488, 'symbol': 'AAPL', 'year': 2024}, {'actual': 1.4, 'estimate': 1.3732, 'period': '2024-06-30', 'quarter': 3, 'surprise': 0.0268, 'surprisePercent': 1.9516, 'symbol': 'AAPL', 'year': 2024}]


In [ ]:
#aggregation of useful data into a dataframe

def data_aggregation():
    symbols="AAPL"
    fromdate="2024-01-01"
    todate ="2024-12-31"
    symbl=get_symbols()
    quote = get_stock_quote(symbols)
    profile = get_company_profile(symbols)
    financials = get_financials(symbols)
    #sentiments = insider_sentiments(symbols,fromdate,todate)
    #eps = eps_surprise(symbols)

    metrics = financials.get("metric", {})

    rows={

        "symbol":symbols,
        "Description":symbl.get("description"),
        "stock_type":symbl.get("type"),
        "Logo":profile.get("logo"),
        "Country":profile.get("country"),
        "currency":profile.get("currency"),
        "Exchange":profile.get("currency"),
        "Industry":profile.get("finnhubIndustry"),
        "market_value":profile.get("marketCapitalization"),
        "current_value":quote.get("c"),
        "Prev_day":quote.get("pc"),
        "day_open":quote.get("o"),
        "High":quote.get("h"),
        "low":quote.get("l"),
        "26w_return": metrics.get("26WeekPriceReturnDaily"),
        "13w_return": metrics.get("13WeekPriceReturnDaily"),
        "ytd_return": metrics.get("yearToDatePriceReturnDaily"),
        "3m_volatility": metrics.get("3MonthADReturnStd"),
        "revenue_growth_5y": metrics.get("revenueGrowth5Y"),
        "revenue_growth_3y": metrics.get("revenueGrowth3Y"),
        "eps_growth_5y": metrics.get("epsGrowth5Y"),
        "eps_growth_3y": metrics.get("epsGrowth3Y"),
        "asset_turnover": metrics.get("assetTurnoverAnnual"),
        "book_value_per_share": metrics.get("bookValuePerShareAnnual"),
        "book_value_growth_5y": metrics.get("bookValueShareGrowth5Y"),
        "dividend_growth_5y": metrics.get("dividendGrowthRate5Y"),
        "cash_per_share": metrics.get("cashPerSharePerShareAnnual"),
        "dividend_per_share": metrics.get("dividendPerShareAnnual"),
        "ebitda_per_share": metrics.get("ebitdPerShareAnnual"),
        "lt_debt_to_equity": metrics.get("longTermDebt/equityAnnual"),
        "total_debt_to_equity": metrics.get("totalDebt/totalEquityAnnual"),
        "operating_margin": metrics.get("operatingMarginAnnual"),
        "market_cap": metrics.get("marketCapitalization"),
        "52w_high": metrics.get("52WeekHigh"),
        "52w_low": metrics.get("52WeekLow"),
        "avg_volume_10d": metrics.get("10DayAverageTradingVolume"),
        "beta": metrics.get("beta"),
        "roe": metrics.get("roeTTM"),
        "roa": metrics.get("roaTTM"),
        "pe_ratio": metrics.get("peNormalizedAnnual"),
        "pb_ratio": metrics.get("pbAnnual")

    }

    return rows



In [ ]:
list_of_symbols = ['AAPL', 'MSFT', 'GOOGL']
